# **Задание, которое в форме для ДЗ - из исходной таблицы требуется удалить строки данных, содержащие попуски, NaN, прочерки и т.д. Столбец Date разделить на три столбца (день, месяц, год), аналогично поступить со столбцом Time (минуты, секунды). Данные в столбце wind_speed_2 оставить только в м/с. В остальных столбцах оставить только числовые данные без лишних символов.**

---

После выполнения работы был замечены некоторые недостатки исходной таблицы:

1. Показания влажности отрицательные;
2. В некоторых ячейках отсутсвует показание в м/с;
3. Была другая задача с исходной таблицей (в задании надо было разделить два стоблца Date и Time, а в таблице всё находилось в столбце Date);
4. При загрузки таблицы в Colab проценты перевелись в числовое соотношение процента от 0 до 1.

Данные недостатки были учтены в данном коде.

---

In [ ]:
# Для использования данных из Drive
from google.colab import drive

#Для работы с данными
import pandas as pd
import numpy as np

# PATH = '/content/drive/MyDrive/Colab Notebooks/s10.xlsx';
# drive.mount('/content/drive');

PATH = '/content/s10.xlsx';

def clear_data(path):

  rew_data = pd.read_excel(path, na_values=['?', '-', ' ', '_', '–',], header = 0);

  data = rew_data.copy();

  data[['DMY', 'HM']] = data['Date'].str.split(' ', 1, expand = True);

  data['Day'] = data['DMY'].str.split('.', expand = True)[0];
  data['Month'] = data['DMY'].str.split('.', expand = True)[1];
  data['Year'] = data['DMY'].str.split('.', expand = True)[2];

  data['Hour'] = data['HM'].str.split(':', expand=True)[0];
  data['Minute'] = data['HM'].str.split(':', expand=True)[1];

  data['Wind_speed'] = data['Wind_speed'].str.split('/', expand = True)[2];

  date = data.replace(['?', '-', ' ', '  ', '   ', '_', '–'], np.nan, inplace = True);
  data = data.dropna(axis=0);

  data = data.replace("[^\d\.]", "", regex = True);

  data["Humidity"] = abs(data.Humidity.astype(float) * 100);

  data = data.drop(labels = ['DMY' ,'HM'], axis=1);

  data_cleared = data.reindex(columns=['Day', 'Month', 'Year', 'Hour', 'Minute',
                                       'Day_number', 'Temp', 'Temp_2', 'Humidity', 'Visibility',
                                       'Lightness', 'Wind_speed',  'Solar_ang_1']);

  convert_dict = {"Day": int, "Month": int, "Year": int, "Hour": int, "Minute": int, "Day_number": int,
                  "Temp": float, "Temp_2": float, "Humidity": float, "Visibility": float, "Lightness": float, "Wind_speed": float, "Solar_ang_1": float};
  data_cleared = data_cleared.astype(convert_dict);

  data_cleared.to_excel('/content/s10_cleared_with_old_index.xlsx');

  data_cleared = data_cleared.reset_index(drop = True);
  data_cleared.to_excel('/content/s10_cleared_with_new_index.xlsx');

  data_cleared.to_excel('/content/s10_cleared_without_index.xlsx', index = False);

  print(f'\tФорматы в таблице:\n{data_cleared.dtypes}\n');

  return data_cleared;

print(f'\tРезультат фильтрации входных данных: \n{clear_data(PATH)}\n')

БЫЛО 
     Temp             Date Day_number  Lightness  Solar_ang_1   Temp_2  \
0    0.45  23.11.2015 0:00       d327      0.604   -20.834944   1.0 °C   
1    0.25  23.11.2015 0:30       d327      0.581   -20.863944   1.0 °C   
2   -0.12  23.11.2015 1:00       d327      0.569   -20.741944   1.0 °C   
3    0.25  23.11.2015 1:30       d327      0.629   -20.835944   1.0 °C   
4    0.17  23.11.2015 2:00       d327      0.551   -20.894944   0.0 °C   
..    ...              ...        ...        ...          ...      ...   
153 -1.38  26.11.2015 4:30       d330      0.595   -21.339679  -4.0 °C   
154 -1.40  26.11.2015 5:00       d330      0.542   -21.391679  -5.0 °C   
155 -1.71  26.11.2015 5:30       d330      0.541   -21.323679  -5.0 °C   
156 -1.47  26.11.2015 6:00       d330      0.577   -21.439679  -5.0 °C   
157 -2.02  26.11.2015 6:30       d330      0.555   -21.399679  -5.0 °C   

     Humidity Visibility           Wind_speed  
0       -0.99    10.0 км  25.2 км/ч / 7.0 m/s  
1        